[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colab-samples/blob/main/notebooks/basic_notebook_features/text_cells.ipynb)




In [ ]:
# Install required libraries
!pip install -U pip setuptools wheel scikit-learn>=1.4 -q
!pip install -U git+https://github.com/pycaret/pycaret.git@master -q
!pip install -U numpy==1.26.4 dask[complete]==2024.4.1 gradio -q
print("✅ All libraries installed!")

In [ ]:
import pandas as pd
import numpy as np  # ⭐ ADD THIS
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# For demo, we'll create synthetic time series with exogenous variables
dates = pd.date_range(start='2020-01-01', periods=365, freq='D')
np.random.seed(42)

# Target: Daily electricity consumption (kWh)
base_consumption = 100
trend = np.linspace(0, 20, 365)
seasonality = 30 * np.sin(2 * np.pi * np.arange(365) / 365)
temperature_effect = np.random.randn(365) * 5  # Temperature variation

# Exogenous variables
temperature = 20 + 10 * np.sin(2 * np.pi * np.arange(365) / 365) + np.random.randn(365) * 3
humidity = 60 + 20 * np.sin(2 * np.pi * np.arange(365) / 180) + np.random.randn(365) * 5
is_weekend = [(d.dayofweek >= 5) * 1 for d in dates]

# Create consumption based on exogenous factors
consumption = base_consumption + trend + seasonality + 0.5 * temperature + 0.2 * humidity + 10 * np.array(is_weekend) + np.random.randn(365) * 5

# Create DataFrame
df = pd.DataFrame({
    'Date': dates,
    'Consumption': consumption,
    'Temperature': temperature,
    'Humidity': humidity,
    'IsWeekend': is_weekend
})

df = df.set_index('Date')

print(f"✅ Energy Dataset loaded: {df.shape}")
print(f"\nTarget: Daily Electricity Consumption (kWh)")
print(f"Exogenous Variables:")
print(f"   • Temperature (°C)")
print(f"   • Humidity (%)")
print(f"   • IsWeekend (0/1)")
print(f"\nTime Range: {df.index.min()} to {df.index.max()}")

# Plot
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
df['Consumption'].plot(ax=axes[0,0], title='Electricity Consumption', color='green')
df['Temperature'].plot(ax=axes[0,1], title='Temperature', color='red')
df['Humidity'].plot(ax=axes[1,0], title='Humidity', color='blue')
df['IsWeekend'].plot(ax=axes[1,1], title='Weekend Indicator', color='purple')
plt.tight_layout()
plt.show()

df.head()

In [ ]:
# Analyze relationships
print("📊 Correlation with Consumption:")
print(df.corr()['Consumption'].sort_values(ascending=False))

print(f"\n📈 Consumption Statistics:")
print(df['Consumption'].describe())

print(f"\n🌡️ Temperature Impact:")
print(f"   Low temp days avg: {df[df['Temperature'] < 15]['Consumption'].mean():.1f} kWh")
print(f"   High temp days avg: {df[df['Temperature'] > 25]['Consumption'].mean():.1f} kWh")

print(f"\n📅 Weekend Effect:")
print(f"   Weekday avg: {df[df['IsWeekend'] == 0]['Consumption'].mean():.1f} kWh")
print(f"   Weekend avg: {df[df['IsWeekend'] == 1]['Consumption'].mean():.1f} kWh")

In [ ]:
from pycaret.time_series import *

# Combine target and exogenous into one DataFrame for PyCaret
# PyCaret will automatically use other columns as features
ts_data = df.copy()

# Initialize time series forecasting
ts_exp = setup(
    data=ts_data,
    target='Consumption',  # Target column
    fh=30,  # Forecast 30 days
    session_id=123,
    fold=3,
    seasonal_period=7,
    verbose=False
)

print("✅ Setup complete!")
print("Note: Temperature, Humidity, and IsWeekend are available as features")

In [ ]:
# Compare models with exogenous variables
print("🔍 Training models with exogenous variables...")
best_models = compare_models(n_select=3, sort='MAE')
print("\n✅ Top 3 models selected")

In [ ]:
best_model = best_models[0]

print(f"📊 Best Model: {best_model.__class__.__name__}")
print("\nGenerating diagnostic plots...")

plot_model(best_model, plot='forecast')
plot_model(best_model, plot='insample')
plot_model(best_model, plot='residuals')

In [ ]:
print("⚡ Tuning hyperparameters...")
tuned_model = tune_model(best_model, n_iter=10, optimize='MAE')

final_model = finalize_model(tuned_model)
print("✅ Final model ready!")

In [ ]:
# Generate predictions
print("🔮 Forecasting next 30 days...")

# Get the last date from the original DataFrame
last_date = df.index.max()

# Define the forecast horizon
forecast_horizon = 30

# Create future dates for the forecast horizon using PeriodIndex
# The start date needs to be converted to a Period object first.
future_dates = pd.period_range(start=(last_date + pd.Timedelta(days=1)).to_period('D'),
                             periods=forecast_horizon, freq='D')

# Create a DataFrame for future exogenous variables
# For simplicity, we'll use the average values for Temperature and Humidity
# and dynamically calculate IsWeekend for the future dates.
mean_temp = df['Temperature'].mean()
mean_humidity = df['Humidity'].mean()

X_future = pd.DataFrame({
    'Temperature': [mean_temp] * forecast_horizon,
    'Humidity': [mean_humidity] * forecast_horizon,
    'IsWeekend': [(d.dayofweek >= 5) * 1 for d in future_dates]
}, index=future_dates)

predictions = predict_model(final_model, fh=forecast_horizon, X=X_future)

print(f"\n✅ Forecast generated!")
print(f"\nFirst 10 predictions:")
print(predictions.head(10))

In [ ]:
model_name = 'energy_forecast_with_exogenous'
save_model(final_model, model_name)

print(f"✅ Model saved as '{model_name}.pkl'")
print(f"📦 Model can be loaded using: loaded = load_model('{model_name}')")

print(f"\n🎯 Time Series Forecasting Summary:")
print(f"   • Dataset: Energy Consumption (365 days)")
print(f"   • Target: Daily Consumption (kWh)")
print(f"   • Exogenous Variables: Temperature, Humidity, IsWeekend")
print(f"   • Best Model: {best_model.__class__.__name__}")
print(f"   • Forecast Horizon: 30 days")
print(f"   • Type: Univariate WITH Exogenous Variables")

In [ ]:
import gradio as gr
from pycaret.time_series import load_model, predict_model

# Load the model
loaded_model = load_model('energy_forecast_with_exogenous')

def forecast_energy(days, temperature, humidity, is_weekend):
    """
    Forecast energy consumption with custom exogenous variables
    """
    try:
        # Get the last date from training data
        last_date = df.index.max()

        # Create future dates as PeriodIndex (matching training data format)
        future_dates = pd.period_range(
            start=(last_date + pd.Timedelta(days=1)).to_period('D'),
            periods=int(days),
            freq='D'
        )

        # Create DataFrame with exogenous variables for future
        X_future = pd.DataFrame({
            'Temperature': [temperature] * int(days),
            'Humidity': [humidity] * int(days),
            'IsWeekend': [1 if is_weekend == "Yes" else 0] * int(days)
        }, index=future_dates)

        # Make predictions with exogenous variables
        predictions = predict_model(loaded_model, fh=int(days), X=X_future)

        # Format output
        result = f"⚡ **Energy Consumption Forecast (WITH Exogenous Variables)**\n\n"
        result += f"📅 Forecasting: {int(days)} days ahead\n"
        result += f"🌡️ Temperature: {temperature}°C\n"
        result += f"💧 Humidity: {humidity}%\n"
        result += f"📆 Day Type: {'Weekend' if is_weekend == 'Yes' else 'Weekday'}\n\n"
        result += f"📊 **Daily Predictions:**\n\n"

        for i, (date, value) in enumerate(zip(predictions.index[:10], predictions['y_pred'][:10]), 1):
            result += f"Day {i} ({date}): **{value:.1f} kWh**\n"

        if int(days) > 10:
            result += f"\n... (showing first 10 of {int(days)} days)\n"

        avg_consumption = predictions['y_pred'].mean()
        total_consumption = predictions['y_pred'].sum()

        result += f"\n📈 **Summary Statistics:**\n"
        result += f"   • Average Daily: **{avg_consumption:.1f} kWh**\n"
        result += f"   • Total Period: **{total_consumption:.1f} kWh**\n"
        result += f"   • Peak Day: **{predictions['y_pred'].max():.1f} kWh**\n"
        result += f"   • Min Day: **{predictions['y_pred'].min():.1f} kWh**\n"

        # Add intelligent insights
        result += f"\n💡 **Insights:**\n"
        if temperature > 28:
            result += f"   • High temp ({temperature}°C) → Increased cooling demand expected\n"
        elif temperature < 10:
            result += f"   • Low temp ({temperature}°C) → Increased heating demand expected\n"
        else:
            result += f"   • Moderate temp ({temperature}°C) → Normal consumption expected\n"

        if is_weekend == "Yes":
            result += f"   • Weekend → Higher residential usage expected\n"
        else:
            result += f"   • Weekday → Normal commercial/industrial load\n"

        if humidity > 75:
            result += f"   • High humidity ({humidity}%) → May affect HVAC efficiency\n"
        elif humidity < 30:
            result += f"   • Low humidity ({humidity}%) → Comfortable conditions\n"

        return result

    except Exception as e:
        return f"❌ Error generating forecast: {str(e)}\n\nTip: Make sure the model has been trained and saved first!"

# Create Gradio interface
demo = gr.Interface(
    fn=forecast_energy,
    inputs=[
        gr.Slider(minimum=1, maximum=30, value=7, step=1, label="📅 Forecast Days"),
        gr.Slider(minimum=-10, maximum=40, value=22, step=1, label="🌡️ Temperature (°C)"),
        gr.Slider(minimum=0, maximum=100, value=60, step=5, label="💧 Humidity (%)"),
        gr.Radio(choices=["Yes", "No"], value="No", label="📆 Weekend?")
    ],
    outputs=gr.Textbox(label="Energy Forecast Results", lines=22),
    title="⚡ Smart Energy Consumption Forecaster",
    description="**Time Series Forecasting WITH Exogenous Variables**\n\nPredict future electricity consumption based on weather conditions and day type. The model uses Temperature, Humidity, and Weekend status to improve forecast accuracy!",
    examples=[
        [7, 25, 70, "No"],   # Warm weekday
        [14, 15, 50, "Yes"],  # Cool weekend
        [30, 35, 80, "No"]    # Hot humid weekday
    ],
    theme="soft"
)

demo.launch(share=True, debug=True)